### 輸入函數及連線到postgreSQL資料庫伺服器

In [1]:
# import streamlit as st
import psycopg2
from dotenv import load_dotenv
import os
load_dotenv()

conn = psycopg2.connect(host     = os.environ['HOST'],
                        database = os.environ['DATABASE'],
                        user     = os.environ['USER'],
                        password = os.environ['PASSWORD'])

### 建立「市場」、「股市」資料庫

In [2]:
sql1 = '''
        CREATE TABLE IF NOT EXISTS 市場(
	        name    VARCHAR(20),
	        country VARCHAR(20),
	        PRIMARY KEY(name),
                UNIQUE(name,country)
                );
        '''

sql2 = '''
        CREATE TABLE IF NOT EXISTS 股市(
        stock_id    SERIAL,
        date        DATE,
        open        NUMERIC(17,10),
        high        NUMERIC(17,10),
        low         NUMERIC(17,10),
        close       NUMERIC(17,10),
        adj_close   NUMERIC(17,10),
        volume      BIGINT DEFAULT 0,
        name        VARCHAR(20),
        PRIMARY KEY(stock_id),
        UNIQUE(date,name),
        FOREIGN KEY(name) REFERENCES 市場(name) 
        ON DELETE NO ACTION
        ON UPDATE CASCADE
        );'''

with conn:
    with conn.cursor() as cursor:        
        cursor.execute(sql1)

    with conn.cursor() as cursor:        
        cursor.execute(sql2)


### 新增台灣、香港恆生至「市場」資料庫

In [3]:
sql3 =  '''
        INSERT INTO 市場 values (%s,%s)
        ON CONFLICT DO NOTHING;
        '''
with conn:
    with conn.cursor() as cursor:
        cursor.execute(sql3,['^TWII','台灣'])
        cursor.execute(sql3,['^HSI','香港恒生'])


### 新增台灣及香港恆生股市資料至「股市」資料庫

In [ ]:
import yfinance as yf
taiwan      = yf.download('^TWII')
hongkong    = yf.download('^HSI')

from decimal import Decimal

sql4 =  '''
        INSERT INTO 股市(date,open,high,low,close,adj_close,volume,name) 
        values(%s,%s,%s,%s,%s,%s,%s,%s)
        ON CONFLICT DO NOTHING;
        '''
with conn:
    all_data        = taiwan.to_dict(orient='index')
    for key,value in all_data.items():
        date        = key.date()
        name        = '^TWII'
        adj_close   = Decimal(value[('Adj Close', name)])
        open        = Decimal(value[('Open'     , name)])
        high        = Decimal(value[('High'     , name)])
        low         = Decimal(value[('Low'      , name)])
        close       = Decimal(value[('Close'    , name)])
        volume      = Decimal(value[('Volume'   , name)])
        with conn.cursor() as cursor:            
            cursor.execute(sql4,[date,open,high,low,close,adj_close,volume,name])

    all_data        = hongkong.to_dict(orient='index')
    for key,value in all_data.items():
        date        = key.date()
        name        = '^HSI'
        adj_close   = Decimal(value[('Adj Close', name)])
        open        = Decimal(value[('Open'     , name)])
        high        = Decimal(value[('High'     , name)])
        low         = Decimal(value[('Low'      , name)])
        close       = Decimal(value[('Close'    , name)])
        volume      = Decimal(value[('Volume'   , name)])
        with conn.cursor() as cursor:            
            cursor.execute(sql4,[date,open,high,low,close,adj_close,volume,name])

conn.close()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
